In [1]:
import numpy as np
from qiskit_optimization import QuadraticProgram
from qiskit import Aer,QuantumRegister, ClassicalRegister, QuantumCircuit, execute
from qiskit.visualization import plot_histogram
from qiskit.algorithms.minimum_eigensolvers import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from itertools import product

In [2]:
from decimal import Decimal
import dimod
from dwave.system import EmbeddingComposite, DWaveSampler
from dwave.system import LeapHybridSampler

In [3]:
def annealer_solver(qubo_matrix,backend="Classical",shots =1000):
    qubo_dict = {}

    for i in range(qubo_matrix.shape[0]):
        for j in range(qubo_matrix.shape[1]):
            if i == j:
                qubo_dict[(i,j)] = qubo_matrix[i,j]
            elif i != j:
                qubo_dict[(i,j)] = qubo_matrix[i,j]
            else:
                pass


    bqm = dimod.BinaryQuadraticModel.from_qubo(qubo_dict)

    if backend == "Classical":
        print("\nPlease wait, while the Classical engine is running....")
        sampleset = dimod.SimulatedAnnealingSampler().sample(bqm, num_reads=shots)

    elif backend == "Quantum":
        print("\nPlease wait, while the Quantum engine is running....")
        sampleset = EmbeddingComposite(DWaveSampler()).sample(bqm, num_reads=shots)

    elif backend == "Hybrid":
        print("\nPlease wait, while the Hybrid Classical-Quantum engine is running....")
        sampleset = LeapHybridSampler().sample(bqm)
    
    else:
        print("\n\nPlease select a valid backend!")
        pass

    best_sample = list(sampleset.first.sample.values())
    return (best_sample)

<h3> Stock List

In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

In [5]:
stocks=['TSLA','AMZN','AAPL','NFLX','GOOG']
start_date = '2020-01-01'
end_date = '2022-01-01'

In [6]:
df=pd.DataFrame()

for stock in stocks:
    df[stock] = yf.Ticker(stock).history(start = start_date, end = end_date, interval = '1d')['Close']

In [ ]:
daily_returns = df.pct_change().dropna()

<h3> QUBO

In [ ]:
mu = daily_returns.mean() * 252
cov = daily_returns.cov() * 252

According to MPT, 

$$\min_{\vec{x}}\,\, -x^{T}\mu + \frac{\gamma}{2} x^T \Sigma x$$

In [13]:
np.diag(mu)

array([[1.53239232, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.33335037, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.50818944, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.3797901 , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.42577824]])

Considering the $L^1$ norm of $x$ vector to be 3 that is,

$$\sum_{i=0}^4 |x_i|=3$$

The associated constraint should be,

$$C=\min_x\left(\sum_{i=0}^4 x_i-3\right)^2=\min_x x^T(J_5-2\cdot 3I_5)x$$

where $J_5$ is a $5\times 5$ matrix with all element 1. 

In [14]:
Jn=np.ones((5,5))
In=np.identity(5)
C=Jn-2*3*In
C

array([[-5.,  1.,  1.,  1.,  1.],
       [ 1., -5.,  1.,  1.,  1.],
       [ 1.,  1., -5.,  1.,  1.],
       [ 1.,  1.,  1., -5.,  1.],
       [ 1.,  1.,  1.,  1., -5.]])

In [15]:
gamma = 1  #risk aversion parameter
qubo = -1*np.diag(mu) + (gamma/2)*np.array(cov) + C
qubo

array([[-6.25598324,  1.05072518,  1.06495328,  1.05482952,  1.04780118],
       [ 1.05072518, -5.28170699,  1.04035328,  1.03901862,  1.03338399],
       [ 1.06495328,  1.04035328, -5.437992  ,  1.03833462,  1.04166983],
       [ 1.05482952,  1.03901862,  1.03833462, -5.30067645,  1.03198419],
       [ 1.04780118,  1.03338399,  1.04166983,  1.03198419, -5.37505837]])

<h3> Solution

In [19]:
solution_dwave_q = annealer_solver(qubo, backend='Quantum')


Please wait, while the Quantum engine is running....


In [20]:
solution_dwave_q 

[1, 0, 1, 0, 1]